In [ ]:
import requests
from bs4 import BeautifulSoup
import datetime
from tqdm import tqdm, tqdm_notebook
import json

def get_date(yyyy, mm, dd):
    g = dd+'/'+mm_eng[mm.capitalize()]+'/'+yyyy
    d = datetime.datetime.strptime(g, '%d/%b/%Y').date()
    return d

def main():
    mm_eng = {'Янв':'Jan', 'Фев':'Feb', 'Мар':'Mar', 'Апр':'Apr',
     'Мая':'May', 'Июн':'Jun', 'Июл':'Jul', 'Авг':'Aug', 
     'Сен':'Sep', 'Окт':'Oct', 'Ноя':'Nov', 'Дек':'Dec'}

    message = ('Внимание, Пользователь! Выполняю запросы на веб-сайт'+
               '\n(анг. "website") посредсвом информационно-телекоммуникационной'+
               '\nсети Интернет (анг. "Internet"). Сохраняйте сопкойствие'+
               '\nи бодрость духа до появления новых указанй или характерного звука.')

    m_r = requests.get('https://career.hse.ru/news/')
    soup = BeautifulSoup(m_r.text, 'lxml')
    num_of_cycles = int(soup.find_all('a', {'class' : 'pages__page'})[-1].text)
    hse_career_posts = []

    print(message)

    with tqdm_notebook(total=num_of_cycles) as pbar:

        for i in range(num_of_cycles):

            r = requests.get(f'https://career.hse.ru/news/page{i}.html')
            soup = BeautifulSoup(r.text, 'lxml')

            posts_content = soup.find_all("div", {"class": "post__content"})
            posts_meta = soup.find_all("div", {"class": "post-meta__date"})

            qq = list(zip(posts_content, posts_meta))

            for e in qq:

                title = e[0].find('a').text
                href = e[0].find('a')['href']
                text_content = e[0].find('div', {'class' : 'post__text'}).text

                dd = e[1].find('div', {'class' : 'post-meta__day'}).text
                mm = e[1].find('div', {'class' : 'post-meta__month'}).text
                yyyy = e[1].find('div', {'class' : 'post-meta__year'}).text

                d = str(get_date(yyyy, mm, dd))

                e_dict = {'title':title, 'href':href, 
                          'text_content':text_content, 'date':d}

                hse_career_posts.append(e_dict)

            pbar.update()

    print("Now we have got all news-posts")  

    print(message)

    with tqdm_notebook(total=len(hse_career_posts)) as pbar:

        for e in hse_career_posts:
            try:
                url = e['href']
                if 'http:' in url:
                    url = url.replace('http:', 'https:')
                if not 'https:' in url:
                    url = 'https:'+url
                r = requests.get(url).text
                e['href_content_html'] = r
            except:
                e['href_content_html'] = ''
                pass

            pbar.update()

    print('Now we catched them all!')

    fin = {'posts':hse_career_posts}

    with open('hse_career_department_website_posts.json', 'w') as f:
        json.dump(hse_career_posts, f)

    print('Done!')

    
main()